In [1]:
import pandas as pd, requests, numpy as np
import re, datetime as dt
from bs4 import BeautifulSoup
import concurrent.futures as thread 

def get_data(_date):
    data2={
        '__VIEWSTATEFIELDCOUNT': 10,
        '__VIEWSTATE': None,
        '__VIEWSTATE1': None,
        '__VIEWSTATE2': None,
        '__VIEWSTATE3': None,
        '__VIEWSTATE4': None,
        '__VIEWSTATE5': None,
        '__VIEWSTATE6': None,
        '__VIEWSTATE7': None,
        '__VIEWSTATE8': None,
        '__VIEWSTATE9': None,
        'ctl00$mdl141$ctl00$Search$txtSearch': None,
        'ctl00$mainContent$ctl03$ddlDate': None,
        '__EVENTVALIDATION': None
    }
    session=requests.Session()
    req=session.get(url,headers=headers)
    soup=BeautifulSoup(req.text,'html.parser')
    lst_get_params=soup.findAll('input',{'type':'hidden','name':re.compile(r'^__VIEWSTATE\d*$|^__EVENTVALIDATION$')})
    for i in range(len(lst_get_params)):
        data2[lst_get_params[i].get('id')]=lst_get_params[i].get('value')
    data2['ctl00$mainContent$ctl03$ddlDate']=_date
    req=session.post(url,data=data2,headers=headers)
    session.close()
    df=pd.read_html(req.content)[0]
    df=df.iloc[:,[1,3,4]].assign(date=pd.to_datetime(_date))
    df.columns=['Mặt hàng','Giá điều chỉnh','Chênh lệch giá','Ngày']
    return df


In [2]:
url=r'https://www.pvoil.com.vn/tin-gia-xang-dau'
headers={
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:92.0) Gecko/20100101 Firefox/92.0'
}
req=requests.get(url,headers=headers)
soup=BeautifulSoup(req.text,'html.parser')
lst_date=[_date.get('value') for _date in soup.findAll('select',{'name':"ctl00$mainContent$ctl03$ddlDate"})[0].findAll('option')[1:]]
with thread.ThreadPoolExecutor() as executor:
    results=[executor.submit(get_data,_date) for _date in lst_date]
    thread.wait(results,timeout=0,return_when=thread.ALL_COMPLETED)
    lst_res=[result.result() for result in results]
pd.concat(lst_res)

,Mặt hàng,Giá điều chỉnh,Chênh lệch giá,Ngày
0,Xăng RON 95-III,22.00,560.00,2022-10-11 15:00:00
1,Xăng E5 RON 92-II,21.29,560.00,2022-10-11 15:00:00
2,"Dầu DO 0,05S-II",24.18,1.98,2022-10-11 15:00:00
3,Dầu KO,22.82,1.14,2022-10-11 15:00:00
0,Xăng RON 95-III,21.44,-1.14,2022-10-03 15:00:00
...,...,...,...,...
3,"Dầu DO 0,005S-IV",17.73,NaN,2018-08-22 00:00:00
4,"Dầu DO 0,05S-II",17.68,NaN,2018-08-22 00:00:00
5,Dầu KO,16.26,NaN,2018-08-22 00:00:00
6,Dầu FO 3%S,NaN,NaN,2018-08-22 00:00:00
